## 4: AgentCore Memory

**What you'll learn:** How Memory enables personalized interactions through Short-Term (conversation context) and Long-Term (persistent facts/preferences) storage.

**Why it matters:** Memory transforms stateless agents into context-aware assistants. Three strategies (USER_PREFERENCE, SEMANTIC, SUMMARY) automatically extract and organize information for future retrieval.

**Real-world value:** Agents remember customer preferences, past issues, and conversation history—enabling continuity across sessions without manual context passing.

**Analogy:** Like a doctor's patient file: recent visit notes (short-term), medical history and allergies (long-term facts), and visit summaries (long-term summaries).

![Memory](images/Memory.png)

---

**Prerequisites:** Completed 1, Memory resource created

### Import Required Libraries

In [ ]:
import time
import uuid
import json
import boto3
from bedrock_agentcore.memory import MemoryClient
from utils.agent_memory import REGION, SESSION_ID, ACTOR_ID

# Get region and memory configuration
session = boto3.Session()
region = session.region_name or 'us-west-2'

# Initialize memory client
memory_client = MemoryClient(region_name=region)

# Load memory configuration from 1
with open('memory_config.json', 'r') as f:
    memory_config = json.load(f)
memory_id = memory_config['memory_id']
print(f"Memory ID: {memory_id}")

# Define a customer ID for this demo
ACTOR_ID = "customer_001"

### Seeding Customer Interaction History

Let's create some historical customer interactions to demonstrate memory. In production, this happens naturally through conversations.

In [ ]:
# Seed previous customer interactions
previous_interactions = [
    ("I bought a Fire TV Stick last week in the US but it's not working properly. Can I return it?", "USER"),
    ("Yes, you can return your Fire TV Stick. According to the US return policy, most items can be returned within 30 days of receipt. Since you purchased it last week, you're well within the return window.", "ASSISTANT"),
    ("I also purchased a Kindle Paperwhite 2 months ago in the UK. Can I still return that?", "USER"),
    ("For the Kindle Paperwhite purchased in the UK 2 months ago, you're outside the standard 30-day return window. However, Kindle devices may have different warranty coverage. Let me check the specific UK policy for you.", "ASSISTANT"),
    ("I prefer shopping for electronics and usually buy Amazon devices. I'm in the US.", "USER"),
    ("Thank you for sharing that! I've noted your preference for Amazon electronics devices and that you're based in the US. This will help me provide more relevant assistance in the future.", "ASSISTANT"),
]

print("📝 Seeding customer interaction history...")
try:
    memory_client.create_event(
        memory_id=memory_id,
        actor_id=ACTOR_ID,
        session_id="previous_session",
        messages=previous_interactions
    )
    print("✅ Customer history seeded successfully!")
    print("⏳ Long-Term Memory processing will begin automatically...")
    print("   This typically takes 20-30 seconds as the system:")
    print("   • Analyzes conversation patterns")
    print("   • Extracts customer preferences (USER_PREFERENCE strategy)")
    print("   • Creates semantic embeddings for facts (SEMANTIC strategy)")
    print("   • Generates conversation summaries (SUMMARY strategy)")
except Exception as e:
    print(f"⚠️ Error seeding history: {e}")

### Waiting for Long-Term Memory Processing

AgentCore Memory processes data asynchronously. Messages are stored immediately in Short-Term Memory, then processed into Long-Term Memory strategies (typically 20-30 seconds).

In [ ]:
# Wait for Long-Term Memory processing
print("🔍 Checking for processed Long-Term Memories...")
retries = 0
max_retries = 6  # 1 minute wait
memories_found = False

while retries < max_retries:
    try:
        # Check for preference memories
        preference_memories = memory_client.retrieve_memories(
            memory_id=memory_id,
            namespace=f"returns/customer/{ACTOR_ID}/preferences",
            query="customer preferences and location"
        )
        
        if preference_memories:
            print(f"✅ Found {len(preference_memories)} preference memories after {retries * 10} seconds!")
            memories_found = True
            break
    except Exception as e:
        print(f"⚠️ Error retrieving memories: {e}")
    
    retries += 1
    if retries < max_retries:
        print(f"⏳ Still processing... waiting 10 more seconds (attempt {retries}/{max_retries})")
        time.sleep(10)

if not memories_found:
    print("⚠️ Memory processing is taking longer than expected. This can happen with system load.")
    print("   You can proceed with the lab - memories will be available shortly.")
else:
    print("\n🎯 AgentCore Memory extracted these customer preferences:")
    print("=" * 80)
    for i, memory in enumerate(preference_memories, 1):
        if isinstance(memory, dict):
            content = memory.get('content', {})
            if isinstance(content, dict):
                text = content.get('text', '')
                if text:
                    print(f"  {i}. {text}")
    
    # Also check semantic memories
    try:
        semantic_memories = memory_client.retrieve_memories(
            memory_id=memory_id,
            namespace=f"returns/customer/{ACTOR_ID}/semantic",
            query="previous return requests and purchases"
        )
        
        if semantic_memories:
            print("\n🧠 AgentCore Memory identified these factual details:")
            print("=" * 80)
            for i, memory in enumerate(semantic_memories, 1):
                if isinstance(memory, dict):
                    content = memory.get('content', {})
                    if isinstance(content, dict):
                        text = content.get('text', '')
                        if text:
                            print(f"  {i}. {text}")
    except Exception as e:
        print(f"⚠️ Error retrieving semantic memories: {e}")
    
    # Check summary memory
    try:
        summary_memories = memory_client.retrieve_memories(
            memory_id=memory_id,
            namespace=f"returns/customer/{ACTOR_ID}/previous_session/summary",
            query="conversation summary"
        )
        
        if summary_memories:
            print("\n📋 AgentCore Memory generated this conversation summary:")
            print("=" * 80)
            for i, memory in enumerate(summary_memories, 1):
                if isinstance(memory, dict):
                    content = memory.get('content', {})
                    if isinstance(content, dict):
                        text = content.get('text', '')
                        if text:
                            print(f"  {i}. {text}")
        else:
            print("\n📋 Conversation Summary:")
            print("=" * 80)
            print("  Summary memory is still processing. This strategy generates")
            print("  conversation summaries and typically takes a bit longer.")
    except Exception as e:
        print(f"⚠️ Error retrieving summary memories: {e}")

### Summary

You've learned how to seed interaction history and observe Long-Term Memory processing across three strategies (USER_PREFERENCE, SEMANTIC, and SUMMARY). AgentCore Memory automatically extracts preferences, facts, and conversation summaries for personalized experiences.

### Next Steps

- **5: AgentCore Runtime** - Deploy your agent to production and test memory-aware responses